<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Uncontrolled-Case" data-toc-modified-id="Uncontrolled-Case-1">Uncontrolled Case</a></span></li><li><span><a href="#MPC-Case" data-toc-modified-id="MPC-Case-2">MPC Case</a></span></li></ul></div>

# Set up the environment

In [1]:
# Import pystorms 
import pystorms 
# Import other Python libraries
import numpy as np
import pandas as pd
from datetime import timedelta

# Overview of Scenario Beta

The `config.yaml` file lists the specifics for `Scenario Beta`. Is is developed based on a Model-Predictive Control method developed and published by [Sadler et al. (2020)](https://doi.org/10.1016/j.jhydrol.2020.124571), and subsequently made available with the [HydroShare collaborative environment](https://doi.org/10.4211/hs.5148675c6a5841e686a3b6aec67a38ee).

## Uncontrolled Case

We run `Scenario Beta` without any control. In this Scenario, we have three control assets: an orifice (`R2`), a pump (`P0`), and a weir (`W0`). For the uncontrolled case, the settings for these three assets are set as follows:

|Control asset |SWMM implementation | Uncontrolled setting | 
|--------------|--------------------|----------------------|
|Orifice 3     |`R2`                | 1.0                  |
|Pump 1        |`P0`                | 0.0                  |
|Weir 1        |`W0`                | 1.0                  |

In [2]:
# run the base uncontrolled instance
env_uncontrolled = pystorms.scenarios.beta()
done = False
actions = np.array([1.0, 0.0, 1.0])
while not done:
    done = env_uncontrolled.step(actions)
uncontrolled_perf = sum(env_uncontrolled.data_log["performance_measure"])


 o  Retrieving project data

The performance for this case is computed to be the following:

In [3]:
print(uncontrolled_perf)

2971770.621866018


## MPC Case

The MPC case has built a controller that is based on the actual time of the simulation. The settings are available in the `mpc_rules_beta.csv` file.

In [4]:
# obtain the controller details from the .csv file with the control settings and corresponding simulation time
mpc_df = pd.read_csv("mpc_rules_beta.csv")
mpc_df['datetime'] = pd.to_datetime(mpc_df['datetime'])
mpc_df
#

,datetime,simtime (hr),R2,P0,W0
0,2016-10-08 00:00:00,0.00,1.000,OFF,1.000
1,2016-10-08 00:15:00,0.25,1.000,OFF,1.000
2,2016-10-08 00:30:00,0.50,0.714,OFF,0.000
3,2016-10-08 00:45:00,0.75,1.000,OFF,0.857
4,2016-10-08 01:00:00,1.00,0.571,ON,0.571
...,...,...,...,...,...
60,2016-10-08 15:00:00,15.00,0.000,OFF,0.000
61,2016-10-08 15:15:00,15.25,0.000,OFF,0.000
62,2016-10-08 15:30:00,15.50,0.000,OFF,0.571
63,2016-10-08 15:45:00,15.75,0.143,ON,0.000


In [5]:
# convert the dataframe of rules to an easily readable dictionary 
mpc_df['P0'].replace({"ON":1.0, "OFF":0.0}, inplace=True)
mpc_df.drop(columns=["simtime (hr)"], inplace=True)
mpc_datetimes = mpc_df['datetime'].to_list()
mpc_controller = mpc_df.set_index('datetime').to_dict()

In [6]:
# initialize the scenario, and obtain the initial controller settings
env_mpc = pystorms.scenarios.beta()
controller_datetime = env_mpc.env.getInitialSimulationDateTime()
actions = np.array([mpc_controller['R2'][controller_datetime], 
                    mpc_controller['P0'][controller_datetime], 
                    mpc_controller['W0'][controller_datetime]])
done = False


 o  Retrieving project data

In [7]:
# Run the simulation
while not done:
    sim_datetime = env_mpc.env.getCurrentSimulationDateTime()
    if (sim_datetime >= controller_datetime) and (controller_datetime in mpc_datetimes):
        actions = np.array([mpc_controller['R2'][controller_datetime], 
                            mpc_controller['P0'][controller_datetime], 
                            mpc_controller['W0'][controller_datetime]])
        controller_datetime += timedelta(minutes=15)
    done = env_mpc.step(actions)
mpc_perf = sum(env_mpc.data_log["performance_measure"])

In [8]:
print(mpc_perf)
#

2925689.498831413


## Results

The results are summarized in the following table:

|                      |Performance | 
|----------------------|---------|
|Uncontrolled instance | 2971770 | 
|MPC controller        | 2925689 |

As expected, the MPC controller performance better than the uncontrolled instance.

## Configuration details for Beta

```yaml
name: beta

states:
        - !!python/tuple 
          - "J33"
          - depthN
        - !!python/tuple
          - "J64"
          - depthN
        - !!python/tuple 
          - "J98"
          - depthN
        - !!python/tuple
          - "J102"
          - depthN
        - !!python/tuple 
          - "OUT0"
          - depthN
        - !!python/tuple
          - "ST0"
          - depthN
        - !!python/tuple 
          - "ST2"
          - depthN

action_space:
        - R2
        - P0
        - W0

performance_targets:
        - !!python/tuple
          - J4
          - flooding
        - !!python/tuple
          - J8
          - flooding
        - !!python/tuple
          - J13
          - flooding          
        - !!python/tuple
          - J33
          - flooding
        - !!python/tuple
          - J53
          - flooding
        - !!python/tuple
          - J54
          - flooding 
        - !!python/tuple
          - J64
          - flooding
        - !!python/tuple
          - J65
          - flooding
        - !!python/tuple
          - J98
          - flooding 
        - !!python/tuple
          - J102
          - flooding
        - !!python/tuple
          - J145
          - flooding
        - !!python/tuple
          - J146
          - flooding
```